In [5]:
#https://keras.io/examples/vision/3D_image_classification/
#!pip install tensorflow

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from loadData import loadData
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
from loadData import loadData

cwd = os.getcwd() + "/archive/SMNI_CMI_TRAIN/*.csv"
EEG_Matrixs = loadData(cwd)

EEG_Matrixs.forward()

Done
Done with forward


In [34]:
import numpy as np

control = np.array(EEG_Matrixs.c_Matrix)
alcoholic = np.array(EEG_Matrixs.a_Matrix)

alcoholic_labels = np.array([1 for _ in range(len(alcoholic))])
control_labels = np.array([0 for _ in range(len(control))])

print(control.shape, alcoholic.shape, sep=" : ")

x_train = np.concatenate((alcoholic[:200], control[:200]), axis=0)
y_train = np.concatenate((alcoholic_labels[:200], control_labels[:200]), axis=0)
x_val = np.concatenate((alcoholic[200:], control[200:]), axis=0)
y_val = np.concatenate((alcoholic_labels[200:], control_labels[200:]), axis=0)

print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

(233, 64, 256) : (233, 64, 256)
Number of samples in train and validation are 400 and 66.


In [35]:
print(x_train.shape, x_val.shape, sep = ' : ')

(400, 64, 256) : (66, 64, 256)


In [43]:
import random

from scipy import ndimage


@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [45]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    #.map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    #.map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [41]:
def get_model(width=400, height=64, depth=256):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=400, height=64, depth=256)
model.summary()

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 400, 64, 256, 1   0         
                             )]                                  
                                                                 
 conv3d_8 (Conv3D)           (None, 398, 62, 254, 64   1792      
                             )                                   
                                                                 
 max_pooling3d_8 (MaxPoolin  (None, 199, 31, 127, 64   0         
 g3D)                        )                                   
                                                                 
 batch_normalization_8 (Bat  (None, 199, 31, 127, 64   256       
 chNormalization)            )                                   
                                                                 
 conv3d_9 (Conv3D)           (None, 197, 29, 125, 64   110656

In [42]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

Epoch 1/100


ValueError: in user code:

    File "/Users/jfarizpevega20@students.desu.edu/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/jfarizpevega20@students.desu.edu/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jfarizpevega20@students.desu.edu/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jfarizpevega20@students.desu.edu/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/jfarizpevega20@students.desu.edu/opt/anaconda3/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        rais